# Proyecto Parcial



In [11]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1746231035121_0002,pyspark,idle,Link,Link,None,✔


In [12]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1746231035121_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1746231035121_0003,pyspark,idle,Link,Link,None,✔


In [33]:
# Importar librerías
from pyspark.sql import SparkSession

# Importar funciones únicas
from pyspark.sql.functions import lit, col

# Importar funciones con un alias, que pudieran 
# tener conflicto con las librerías base de Python, 
# como sum, avg, round, ...
import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, StringType, TimestampType, 
    StringType, StringType)


raw_def_columns = StructType([
    StructField("producto", StringType(), True),
    StructField("presentacion", StringType(), True),
    StructField("marca", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("catalogo", StringType(), True),
    StructField("precio", StringType(), True),
    StructField("fecha", StringType(), True),
    StructField("tienda", StringType(), True),
    StructField("tipo_tienda", LongType(), True),
    StructField("sucursal", StringType(), True),
    StructField("direccion", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("ciudad", StringType(), True),
    StructField("latitud", IntegerType(),True),
    StructField("longitud", IntegerType(), True)
]) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Recuerda que vamos a trabajar con spark dataframes, por lo tanto tienes que abrir un SparkSession

In [35]:
spark._sc.install_pypi_package("pandas")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1746231035121_0003/container_1746231035121_0003_01_000001/pyspark.zip/pyspark/context.py", line 2614, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [36]:
# bucket
NAME = 'thmrudolf' ##CAMBIAR AQUÍ SU NOMBRE.
BUCKET = f"s3://itam-analytics-{NAME}"
FOLDER = 'profeco'


# type of catalog
CATALOG_TYPE = 'basicos'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
#Descarga de datos de S3
raw_df = spark.read.csv(f"{BUCKET}/{FOLDER}/raw_descomprimidos/*/*.csv", 
                        header=True, 
                        inferSchema=True, 
                       schema=raw_def_columns)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
raw_df.printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: string (nullable = true)
 |-- fecha: string (nullable = true)
 |-- tienda: string (nullable = true)
 |-- tipo_tienda: long (nullable = true)
 |-- sucursal: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- latitud: integer (nullable = true)
 |-- longitud: integer (nullable = true)

In [51]:

#Cambiamos tipo a variable fecha
# Convertimos columnas a tipo correcto
from pyspark.sql.functions import to_date, col


raw_df = raw_df.withColumn("fecha", to_date(col("fecha"), "yyyy-MM-dd"))
# Revisamos que esté todo en orden
print(raw_df.dtypes)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('producto', 'string'), ('presentacion', 'string'), ('marca', 'string'), ('tipo', 'string'), ('catalogo', 'string'), ('precio', 'string'), ('fecha', 'date'), ('tienda', 'string'), ('tipo_tienda', 'bigint'), ('sucursal', 'string'), ('direccion', 'string'), ('estado', 'string'), ('ciudad', 'string'), ('latitud', 'int'), ('longitud', 'int')]

In [54]:
#Eliminamos las columnas dirección, latitud y longitud
raw_df.drop('direccion', 'latitud', 'longitud')

print(raw_df.dtypes)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('producto', 'string'), ('presentacion', 'string'), ('marca', 'string'), ('tipo', 'string'), ('catalogo', 'string'), ('precio', 'string'), ('fecha', 'date'), ('tienda', 'string'), ('tipo_tienda', 'bigint'), ('sucursal', 'string'), ('direccion', 'string'), ('estado', 'string'), ('ciudad', 'string'), ('latitud', 'int'), ('longitud', 'int')]

In [55]:
#Convertimos el contenido a minúsculas for col in raw_df.columns: 

from pyspark.sql.functions import lower, col

# Convert all string columns to lowercase
for column in raw_df.columns:
    raw_df = raw_df.withColumn(column, lower(col(column)))

raw_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+--------------------+-----------------+--------+----------+---------+-----------+--------------------+--------------------+--------------+--------------+-------+--------+
|          producto|        presentacion|             marca|                tipo|         catalogo|  precio|     fecha|   tienda|tipo_tienda|            sucursal|           direccion|        estado|        ciudad|latitud|longitud|
+------------------+--------------------+------------------+--------------------+-----------------+--------+----------+---------+-----------+--------------------+--------------------+--------------+--------------+-------+--------+
|         batidoras|62647. manual. 6 ...|    hamilton beach| aparatos electricos|electrodomesticos|  799.00|2018-07-09|liverpool|       NULL|liverpool sucursa...|zacatecas nte. 85...|aguascalientes|aguascalientes|   NULL|    NULL|
|         batidoras|2532 ó 2532-013. ...|             oster| aparatos electr

In [56]:
#Eliminamos acentos en las columnas
from pyspark.sql.functions import regexp_replace, col

# List of string columns to normalize
string_columns = [field.name for field in raw_df.schema.fields if field.dataType.simpleString() == "string"]

# Apply normalization
for column in string_columns:
    raw_df = raw_df.withColumn(column, regexp_replace(col(column), "[^\\x00-\\x7F]", ""))

raw_df.show(5)  # Show first 5 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------+-------------------+-----------------+-------+----------+---------+-----------+--------------------+--------------------+--------------+--------------+-------+--------+
| producto|        presentacion|         marca|               tipo|         catalogo| precio|     fecha|   tienda|tipo_tienda|            sucursal|           direccion|        estado|        ciudad|latitud|longitud|
+---------+--------------------+--------------+-------------------+-----------------+-------+----------+---------+-----------+--------------------+--------------------+--------------+--------------+-------+--------+
|batidoras|62647. manual. 6 ...|hamilton beach|aparatos electricos|electrodomesticos| 799.00|2018-07-09|liverpool|       NULL|liverpool sucursa...|zacatecas nte. 85...|aguascalientes|aguascalientes|   NULL|    NULL|
|batidoras|2532  2532-013. m...|         oster|aparatos electricos|electrodomesticos| 699.00|2018-07-09|liverpool|       NULL|liverpool 

In [57]:
#Subimos los datos a S3 en formato parquet
raw_df.repartition("catalogo") \
  .write.mode("overwrite") \
  .partitionBy("catalogo") \
  .option("compression", "snappy") \
  .parquet(f"{BUCKET}/{FOLDER}/parquet")
# Comprobamos que se haya subido correctamente
df = spark.read.parquet(f"{BUCKET}/{FOLDER}/parquet")
df.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+---------+--------------------+------+----------+---------------+-----------+--------------------+--------------------+--------------+--------------+-------+--------+--------+
|   producto|        presentacion|    marca|                tipo|precio|     fecha|         tienda|tipo_tienda|            sucursal|           direccion|        estado|        ciudad|latitud|longitud|catalogo|
+-----------+--------------------+---------+--------------------+------+----------+---------------+-----------+--------------------+--------------------+--------------+--------------+-------+--------+--------+
|     aceite| botella 1 lt. mixto|    1-2-3|aceites y grasas ...| 23.87|2018-07-09|   mega soriana|       NULL|mega soriana sucu...|aguascalientes nt...|aguascalientes|aguascalientes|   NULL|    NULL| basicos|
|desodorante|barra 45 gr. tapa...|     dove|arts. para el cui...| 41.75|2018-09-27|mercado soriana|       NULL|mercado soriana s...|blvd. diaz ordaz ...|       